In [57]:
import joblib
import os
import boto3
import sagemaker
import pandas as pd
from sklearn.metrics import accuracy_score

from sagemaker import get_execution_role

region = boto3.Session().region_name

role = get_execution_role()

In [4]:
bucket = 'crop-reco'
bucket_path = "https://s3-{}.amazonaws.com/{}".format(region, bucket)

In [5]:
print(role)
print(region)
print(bucket)
print(bucket_path)

arn:aws:iam::799480794502:role/service-role/AmazonSageMaker-ExecutionRole-20220301T182533
us-east-2
crop-reco
https://s3-us-east-2.amazonaws.com/crop-reco


In [6]:
model_file_name = "latest-model"

In [7]:
model = joblib.load(model_file_name)

In [ ]:
df_test = pd.read_csv()

In [8]:
model._Booster.save_model(model_file_name)
!tar czvf model.tar.gz $model_file_name

latest-model


In [9]:
#### prefix in S3
prefix = "crop/deployement"

fObj = open("model.tar.gz", "rb")
key = os.path.join(prefix, model_file_name, "model.tar.gz")
print(key)
boto3.Session().resource("s3").Bucket(bucket).Object(key).upload_fileobj(fObj)

crop/deployement/latest-model/model.tar.gz


In [10]:
from sagemaker.amazon.amazon_estimator import get_image_uri

container = get_image_uri(boto3.Session().region_name, "xgboost", "1.3-1")

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [11]:
%%time
from time import gmtime, strftime

model_name = model_file_name + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

model_url = "https://s3-{}.amazonaws.com/{}/{}".format(region, bucket, key)

sm_client = boto3.client("sagemaker")

print(model_url)

primary_container = {
    "Image": container,
    "ModelDataUrl": model_url,
}

create_model_response2 = sm_client.create_model(
    ModelName=model_name, ExecutionRoleArn=role, PrimaryContainer=primary_container
)

print(create_model_response2["ModelArn"])

https://s3-us-east-2.amazonaws.com/crop-reco/crop/deployement/latest-model/model.tar.gz
arn:aws:sagemaker:us-east-2:799480794502:model/latest-model2022-03-01-13-59-28
CPU times: user 62 ms, sys: 4.55 ms, total: 66.6 ms
Wall time: 414 ms


In [12]:
from time import gmtime, strftime

endpoint_config_name = "crop-XGBoostEndpointConfig-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

print(endpoint_config_name)

create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "InstanceType": "ml.m4.xlarge",
            "InitialInstanceCount": 1,
            "InitialVariantWeight": 1,
            "ModelName": model_name,
            "VariantName": "AllTraffic",
        }
    ],
)

print("Endpoint Config Arn: " + create_endpoint_config_response["EndpointConfigArn"])

crop-XGBoostEndpointConfig-2022-03-01-13-59-50
Endpoint Config Arn: arn:aws:sagemaker:us-east-2:799480794502:endpoint-config/crop-xgboostendpointconfig-2022-03-01-13-59-50


In [31]:
%%time
import time

endpoint_name = "crop-reco-Endpoint-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_name)
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name
)
print(create_endpoint_response["EndpointArn"])

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

crop-reco-Endpoint-2022-03-01-14-28-38
arn:aws:sagemaker:us-east-2:799480794502:endpoint/crop-reco-endpoint-2022-03-01-14-28-38
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: InService
Arn: arn:aws:sagemaker:us-east-2:799480794502:endpoint/crop-reco-endpoint-2022-03-01-14-28-38
Status: InService
CPU times: user 57.3 ms, sys: 16.8 ms, total: 74.1 ms
Wall time: 4min


In [32]:
runtime_client = boto3.client('runtime.sagemaker')

In [36]:
%%time
import json


file_name = "test_point.csv"


with open(file_name, "r") as f:
    next(f)
    payload = f.read()
    
# print(payload)

-1.0713617175044392,0.7469035442380891,-0.5162913988163972,-0.3394078335155689,-2.1891125538540117,-0.6834313090538562,0.1673627690169279
-1.0442741709198693,-0.4354204308113193,-0.5360355421447951,0.6668868016567626,-0.3872785543941541,2.2151199046885743,-0.6003151111469268
-0.7463111584896003,0.11026755767302301,-0.5755238288015909,1.4264913780038695,-0.15647560947688915,1.0097979925658602,-0.5469434068679302
0.25592806513948635,0.14058355703326425,-0.22012924889042867,0.1031787514539556,0.3430281940458805,-0.5196515978956271,1.2377655115094657
0.4455408912314757,-0.3747884320908368,-0.16089681890523494,0.045924702545929376,0.7336945673635562,1.1159336896257883,0.8944182353887665
-0.3670855063056216,0.6256395467971241,-0.6545004021151825,-0.1367845970444023,-2.3920374439513448,-1.1464768766011064,-0.7415562866680427
-0.04203494729078265,0.04963555895254052,0.09577704436393776,2.5328326624123254,0.9951400766615645,0.09632521053910988,-0.4684996645073175
-1.2338869970118589,-1.04174041

In [37]:
response = runtime_client.invoke_endpoint(
    EndpointName=endpoint_name, ContentType="text/csv", Body=payload
)



print("Results :\n")
print()

result = response["Body"].read().decode("ascii")

# print("\nPredicted Class Probabilities: {}.".format(result))

Results :




In [38]:
result_lol = '['+result+']'

In [39]:
import ast
final_result_lol = ast.literal_eval(result_lol)

In [40]:
len(final_result_lol)

660

In [41]:
# final_result_lol

[[0.0001742954336805269,
  0.0004053190059494227,
  0.0002066957240458578,
  0.00019084390078205615,
  0.00014886670396663249,
  0.00016768064233474433,
  0.00020216079428792,
  0.00017847129493020475,
  0.0001099338915082626,
  0.9930146932601929,
  0.000278595311101526,
  0.0003808398905675858,
  0.00041112510371021926,
  0.000218799221329391,
  0.00017436443886253983,
  0.00023866249830462039,
  0.00018443851149640977,
  0.00014323621871881187,
  0.0027548433281481266,
  0.00016826213686726987,
  0.00010571088932920247,
  0.0001421294000465423],
 [0.00019301357679069042,
  0.00019280439300928265,
  0.004138699267059565,
  0.0002113396185450256,
  0.00016485412197653204,
  0.0002890996402129531,
  0.00018025896861217916,
  0.0002181919990107417,
  0.00010853236017283052,
  0.00020617870904970914,
  0.00028525065863505006,
  0.00038862056680954993,
  0.0002450084430165589,
  0.9914400577545166,
  0.00029284460470080376,
  0.00026429322315379977,
  0.00020424633112270385,
  0.000201068

In [42]:
endpoint_pred = []
for i in final_result_lol:
    max_value = max(i)
    endpoint_pred.append(i.index(max_value))

In [43]:
endpoint_pred

[9,
 13,
 2,
 8,
 8,
 9,
 17,
 12,
 5,
 11,
 20,
 17,
 4,
 16,
 12,
 1,
 10,
 3,
 2,
 6,
 12,
 15,
 1,
 9,
 18,
 17,
 8,
 12,
 9,
 7,
 13,
 8,
 14,
 19,
 13,
 20,
 9,
 6,
 2,
 7,
 13,
 0,
 4,
 21,
 6,
 9,
 11,
 3,
 16,
 6,
 12,
 9,
 15,
 10,
 0,
 1,
 0,
 11,
 1,
 4,
 10,
 3,
 1,
 18,
 4,
 20,
 18,
 17,
 5,
 20,
 16,
 21,
 5,
 1,
 3,
 8,
 15,
 18,
 11,
 2,
 7,
 6,
 7,
 16,
 3,
 2,
 18,
 20,
 5,
 4,
 13,
 21,
 7,
 7,
 1,
 7,
 0,
 19,
 14,
 17,
 8,
 1,
 5,
 15,
 18,
 3,
 7,
 21,
 19,
 17,
 16,
 13,
 19,
 7,
 8,
 21,
 12,
 20,
 6,
 13,
 14,
 2,
 16,
 7,
 15,
 12,
 4,
 6,
 13,
 3,
 19,
 19,
 16,
 9,
 9,
 8,
 14,
 21,
 18,
 0,
 11,
 12,
 16,
 5,
 17,
 5,
 8,
 7,
 12,
 16,
 8,
 9,
 15,
 19,
 9,
 10,
 10,
 17,
 19,
 9,
 11,
 10,
 2,
 4,
 16,
 15,
 5,
 9,
 10,
 21,
 20,
 3,
 7,
 13,
 0,
 21,
 21,
 18,
 14,
 8,
 5,
 17,
 11,
 8,
 8,
 1,
 13,
 3,
 11,
 4,
 2,
 18,
 5,
 17,
 12,
 21,
 15,
 15,
 21,
 15,
 0,
 2,
 16,
 13,
 21,
 19,
 4,
 20,
 15,
 11,
 9,
 15,
 14,
 0,
 11,
 12,
 9,
 19,
 13,
 0,
 7

In [45]:
d = {}
for index, value in enumerate(list(model.classes_)):
    d[index] = value

In [49]:
predictions = []
for i in endpoint_pred:
    predictions.append(d[i])

In [58]:
accuracy_score(pd.read_csv('df_y_test.csv'),predictions)

0.9924242424242424

In [59]:
sm_client.delete_endpoint(EndpointName=endpoint_name)

{'ResponseMetadata': {'RequestId': '6f32f8f8-d80f-4a87-be2f-e2a81783d587',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '6f32f8f8-d80f-4a87-be2f-e2a81783d587',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Tue, 01 Mar 2022 14:53:40 GMT'},
  'RetryAttempts': 0}}